In [17]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import pandas
from numba import jit
from scipy.integrate import odeint

In [18]:
#Construct a model
@jit(nopython=True)
def model(u, t, p):
    k, big_k, r_ifn_v, d_v, p_v_ifn, d_ifn, k1, k2, d_mcp1, n1, mcp10 = p
    v, ifn, mcp1 = u
    dy = (k*v*(1-v/big_k) - r_ifn_v*(ifn)*v - d_v*v,
    p_v_ifn*v - d_ifn*(ifn),
    (k1*(ifn)**n1)/(k2+(ifn)**n1)-(mcp1-mcp10)*d_mcp1)
    return dy

In [91]:
def traj(noise):
    #timepoints for solver
    tSol = (0, 0.125, 0.25, 0.375, 0.5, 0.75, 1, 1.25, 1.5, 2, 2.5, 3, 5)

    #Initial conditions
    u01 = [6.819798, 0, 121.7883333] #h1n1
    u05 = [7.10321, 0, 121.525] #h5n1

    #Parameter values
    #h1n1
    par1 = np.array((3.27, 10.9, 0.00016, 0.000201, 0.1, 1.08, 14700, 18500, 1.04, 5.45))
    mcp101 = 121.7883333

    #h5n1
    par5 = np.array((3.27, 10.9, 0.00016, 0.000201, 0.4, 1.08, 14700, 18500, 1.04, 5.45))
    mcp105 = 121.525

    #Add noise to the parameters, then remove zeros
    parN1 = np.random.normal(par1,i*par1)
    parN1[parN1<0] = 1e-4

    parN5 = np.random.normal(par5,i*par5)
    parN5[parN5<0] = 1e-4

    #Prepare arguments for solver
    argTup1 = np.append(parN1,mcp101)
    argTup5 = np.append(parN5,mcp105)

    #Solve the model
    sol1 = odeint(model,u01,tSol,args=(argTup1,),mxstep=5000)
    sol5 = odeint(model,u05,tSol,args=(argTup5,),mxstep=5000)

    #Put MCP1 in log2 space
    sol1[:,2] = np.log2(sol1[:,2])
    sol5[:,2] = np.log2(sol5[:,2])
    
    out1 = np.expand_dims(sol1,axis=-1)
    out5 = np.expand_dims(sol5,axis=-1)
    
    out = np.concatenate((out1,out5),-1)
    
    return out